In [ ]:
### Pré-processamento do dataset - Ataques ###
import pandas as pd

ataques = ['ackscan','bruteforcedirb','cmsscan','dosddos','fullconnectscan','icmpechodiscover','mysqlbruteforces','nikto','scanvuln','ssh_a','stealthscan','synscan','udpscan','wapiti']

path_base = 'dataset_base'
path_in = "dataset/in"

for ataque in ataques:
    
    print(ataque)

    # Carregar um dataset (exemplo usando um arquivo CSV)
    df = pd.read_csv(path_base + '/ataque/'+ataque+'.csv')
    
    # Exibir as primeiras 5 linhas do dataset
    print("Antes de dropar colunas:")
    print(df.head())
    
    # Dropar colunas específicas
    colunas_para_remover = ['Process Time', 'Src IP', 'Src Port', 'Dst IP', 'Dst Port', 'Previous Flow Time', 'DSCP']
    
    if 'Svc Type' in df.columns:
        colunas_para_remover = ['Process Time', 'Src IP', 'Src Port', 'Dst IP', 'Dst Port', 'Previous Flow Time', 'DSCP', 'Svc Type']
     
    df = df.drop(columns=colunas_para_remover)

    df['Label'] = 'ataque'
    df = df.drop(ataque, axis=1)
    
    # Exibir as primeiras 5 linhas do dataset após dropar colunas
    print("\nDepois de dropar colunas:")
    print(df.head())
    
    df.to_csv(path_in + '/ataque/'+ataque+'.csv', index=False)
    
    print("Dataset salvo com sucesso!")


In [ ]:
### Pré-processamento do dataset - Normal ###
import pandas as pd

normais = ['dns','http','smtp','snmp','ssh']

path_base = 'dataset_base' 
path_in = "dataset/in"

for normal in normais:

    print(normal)

    # Carregar um dataset (exemplo usando um arquivo CSV)
    df = pd.read_csv(path_base + '/normal/'+normal+'.csv')
    
    # Exibir as primeiras 5 linhas do dataset
    print("Antes de dropar colunas:")
    print(df.head())
    
    # Dropar colunas específicas
    colunas_para_remover = ['Process Time', 'Src IP', 'Src Port', 'Dst IP', 'Dst Port', 'Previous Flow Time', 'DSCP']
    
    if 'Svc Type' in df.columns:
        colunas_para_remover = ['Process Time', 'Src IP', 'Src Port', 'Dst IP', 'Dst Port', 'Previous Flow Time', 'DSCP', 'Svc Type']
     
    df = df.drop(columns=colunas_para_remover)
    
    # Exibir as primeiras 5 linhas do dataset após dropar colunas
    print("\nDepois de dropar colunas:")
    print(df.head())
    
    df.to_csv(path_in + '/normal/'+normal+'.csv', index=False)
    
    print("Dataset salvo com sucesso!")


In [7]:
def criarDataset(path_in, path_out, evento, classe):

    data = pd.read_csv(path_in + '/'+ classe +'/'+ evento +'.csv')

    # Dataset 40% 30% 30%
    
    # Dividindo o dataset em 40% treino e 60% restante (teste + validação)
    train_data, temp_data = train_test_split(data, test_size=0.6, random_state=16)
    
    # Dividindo o restante (60%) em 30% teste e 30% validação
    test_data, val_data = train_test_split(temp_data, test_size=0.5, random_state=16)
    
    train_data.to_csv(path_out + '/'+ evento +'/train.csv', index=False)
    test_data.to_csv(path_out + '/'+ evento +'/test.csv', index=False)
    val_data.to_csv(path_out + '/'+ evento +'/val.csv', index=False)
    
    print("Train: ",train_data.shape)
    print("Test: ",train_data.shape)
    print("Val: ",train_data.shape)


In [11]:
# Combinação dos ataques - Aleatório
import random
import pandas as pd
from sklearn.model_selection import train_test_split
import os

# Suas listas
lista_ataques = ['ackscan','bruteforcedirb','cmsscan','dosddos','fullconnectscan','icmpechodiscover','mysqlbruteforces','nikto','scanvuln','ssh_a','stealthscan','synscan','udpscan','wapiti']

lista_normais = ['dns','http','smtp','snmp','ssh']

path_in = "dataset/in"
path_out = "dataset/out"

for ataque in lista_ataques:
    print(ataque)
    classe = 'ataque'
    os.mkdir(path_out+'/ataque/'+ataque)
    criarDataset(path_in, path_out+'/ataque', ataque)
    print("\n")

for normal in lista_normais:
    print(normal)
    classe = 'normal'
    os.mkdir(path_out+'/normal/'+normal)
    criarDataset(path_in, path_out+'/normal', normal, classe)
    print("\n")



Train:  (19671, 41)
Test:  (19671, 41)
Val:  (19671, 41)
Train:  (206610, 41)
Test:  (206610, 41)
Val:  (206610, 41)
Train:  (114735, 41)
Test:  (114735, 41)
Val:  (114735, 41)
Train:  (1106, 41)
Test:  (1106, 41)
Val:  (1106, 41)
Train:  (7314, 41)
Test:  (7314, 41)
Val:  (7314, 41)


In [31]:
def concatenarDatasetTrain(path_in, path_out, indice, lista):

    with open(path_out + '/exp'+ str(indice + 1) +'/train.txt', "w") as arquivo:
        arquivo.write(str(lista))

    df1 = pd.read_csv(path_in + '/ataque/'+ lista[0] +'/train.csv')
    df2 = pd.read_csv(path_in + '/ataque/'+ lista[1] +'/train.csv')
    df3 = pd.read_csv(path_in + '/ataque/'+ lista[2] +'/train.csv')
    df4 = pd.read_csv(path_in + '/ataque/'+ lista[3] +'/train.csv')
        
    df_malware = pd.concat([df1, df2, df3, df4], ignore_index=True)
    
    df5 = pd.read_csv(path_in + '/normal/'+ lista[4] +'/train.csv')
    df6 = pd.read_csv(path_in + '/normal/'+ lista[5] +'/train.csv')
    
    df_normal = pd.concat([df5, df6], ignore_index=True)
    
    print("Dimensao normal: ",df_normal.shape)
    print("Dimensao malware: ",df_malware.shape)
    
    # Realizando undersampling (malware tem um número maior de samples)
    df_malware = df_malware.sample(len(df_normal))
    print("Samples malware balanceado: ",df_malware.shape[0])

    if df_normal.shape[0] < df_malware.shape[0]:
        df_malware = df_malware.sample(len(df_normal))
    else:
        df_normal = df_normal.sample(len(df_malware))        
    
    # Combinando as classes balanceadas
    df_balanceado = pd.concat([df_malware, df_normal])
    
    # Embaralhando as instâncias
    train_data = df_balanceado.sample(frac=1).reset_index(drop=True)
    
    # Exibindo o dataset balanceado
    print("Samples dataset balanceado - train: ",train_data.shape[0])
    
    train_data.to_csv(path_out + '/exp'+ str(indice + 1) +'/train.csv', index=False)
    print("Train: ",train_data.shape)



In [33]:
def concatenarDatasetTest(path_in, path_out, indice, lista):
    # ataques
    df1 = pd.read_csv(path_in + '/ataque/'+ lista[0] +'/test.csv')
    df2 = pd.read_csv(path_in + '/ataque/'+ lista[1] +'/test.csv')
    df3 = pd.read_csv(path_in + '/ataque/'+ lista[2] +'/test.csv')
    df4 = pd.read_csv(path_in + '/ataque/'+ lista[3] +'/test.csv')
    df5 = pd.read_csv(path_in + '/ataque/'+ lista[4] +'/test.csv')
    df6 = pd.read_csv(path_in + '/ataque/'+ lista[5] +'/test.csv')
    df7 = pd.read_csv(path_in + '/ataque/'+ lista[6] +'/test.csv')
    df8 = pd.read_csv(path_in + '/ataque/'+ lista[7] +'/test.csv')
    df9 = pd.read_csv(path_in + '/ataque/'+ lista[8] +'/test.csv')
    df10 = pd.read_csv(path_in + '/ataque/'+ lista[9] +'/test.csv')

    # normais
    df11 = pd.read_csv(path_in + '/normal/'+ lista[10] +'/test.csv')
    df12 = pd.read_csv(path_in + '/normal/'+ lista[11] +'/test.csv')
    df13 = pd.read_csv(path_in + '/normal/'+ lista[12] +'/test.csv')
    
    test_data = pd.concat([df1, df2, df3, df4, df5, df6, df7, df8, df9, df10, df11, df12, df13 ], ignore_index=True)
    
    # Exibindo o dataset balanceado
    print("Samples dataset - test: ",test_data.shape[0])
    
    test_data.to_csv(path_out + '/exp'+ str(indice + 1) +'/test.csv', index=False)
    print("Test: ",test_data.shape)



In [35]:
# Combinação dos ataques - Aleatório
import random
import pandas as pd
import os

# Suas listas
lista_ataques = ['ackscan','bruteforcedirb','cmsscan','dosddos','fullconnectscan','icmpechodiscover','mysqlbruteforces','nikto','scanvuln','ssh_a','stealthscan','synscan','udpscan','wapiti']

lista_normais = ['dns','http','smtp','snmp','ssh']

path_in = "dataset/out"
path_out = "dataset/combination"

for i in range(8):
    
    #os.mkdir(path_out+'/exp'+str(i+1))
    
    # Selecionando aleatoriamente 4 ataques e 2 normais
    train_ataques = random.sample(lista_ataques, 4)
    train_normais = random.sample(lista_normais, 2)
    
    # Combinando as seleções em uma nova lista
    train_lista = train_ataques + train_normais
    
    test_ataques = lista_ataques.copy()
    test_normais = lista_normais.copy()

    
    # Separar samples de teste
    for item in train_ataques:
        # remove ataques adiconados para treino
        test_ataques.remove(item)

    for item in train_normais:
        # remove os eventos adicionados para treino
        test_normais.remove(item)

    test_lista = test_ataques + test_normais

    print(train_lista)
    print(test_lista)    
    
    concatenarDatasetTrain(path_in, path_out, i, train_lista)
    concatenarDatasetTest(path_in, path_out, i, test_lista)
    

['udpscan', 'ackscan', 'icmpechodiscover', 'dosddos', 'http', 'snmp']
['bruteforcedirb', 'cmsscan', 'fullconnectscan', 'mysqlbruteforces', 'nikto', 'scanvuln', 'ssh_a', 'stealthscan', 'synscan', 'wapiti', 'dns', 'smtp', 'ssh']
Dimensao normal:  (207716, 41)
Dimensao malware:  (10905126, 41)
Samples malware balanceado:  207716
Samples dataset balanceado - train:  415432
Train:  (415432, 41)
Samples dataset - test:  17107147
Test:  (17107147, 41)
['cmsscan', 'ssh_a', 'stealthscan', 'dosddos', 'smtp', 'ssh']
['ackscan', 'bruteforcedirb', 'fullconnectscan', 'icmpechodiscover', 'mysqlbruteforces', 'nikto', 'scanvuln', 'synscan', 'udpscan', 'wapiti', 'dns', 'http', 'snmp']
Dimensao normal:  (122049, 41)
Dimensao malware:  (6029088, 41)
Samples malware balanceado:  122049
Samples dataset balanceado - train:  244098
Train:  (244098, 41)
Samples dataset - test:  20828424
Test:  (20828424, 41)
['ssh_a', 'icmpechodiscover', 'bruteforcedirb', 'nikto', 'ssh', 'dns']
['ackscan', 'cmsscan', 'dosddos'

In [15]:
import pandas as pd
# => Este código é para criar os datasets de teste por eventos por Dataset de 1 ao 8
path_in = 'dataset/out/'
path_out = 'dataset/combination/'
indice = 8
lista = ['udpscan', 'dosddos', 'stealthscan', 'scanvuln', 'http', 'ssh']

# ataques
df1 = pd.read_csv(path_in + '/ataque/'+ lista[0] +'/test.csv')
df2 = pd.read_csv(path_in + '/ataque/'+ lista[1] +'/test.csv')
df3 = pd.read_csv(path_in + '/ataque/'+ lista[2] +'/test.csv')
df4 = pd.read_csv(path_in + '/ataque/'+ lista[3] +'/test.csv')

# normais
df5 = pd.read_csv(path_in + '/normal/'+ lista[4] +'/test.csv')
df6 = pd.read_csv(path_in + '/normal/'+ lista[5] +'/test.csv')

test_data = pd.concat([df1, df2, df3, df4, df5, df6], ignore_index=True)
    
test_data.to_csv(path_out + '/exp'+ str(indice) +'/test_evento.csv', index=False)